## 자전거 대여 수요 예측
#### 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재되어 있다.

- datetime: hourly date + timestamp
- season : 1=봄, 2=여름, 3=가을, 4=겨울
- holiday : 1=토,일요일의 주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday : 1=토,일요일의 주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
 - 1=맑음, 약간 구름 낀 흐림
 - 2=안개, 안개+흐림
 - 3=가벼운 눈, 가벼운 비+천둥
 - 4=심한 눈/비, 천둥/번개
- temp : 온도 (섭씨)
- atemp : 체감온도 (섭씨)
- humidity : 상대습도
- windspeed : 풍속
- casual : 사전에 등록되지 않은 사용자가 대여한 횟수
- registered : 사전에 등록된 사용자가 대여한 횟수
- count : 대여 횟수

## 다음 코드는 복습과제를 하는데 있어 필요한 모듈입니다. 한 번 꼭 돌려주고 실습해 주세요.

In [127]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta

In [128]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

## 1. 데이터 불러오기 및 전처리
### (1) bikeshare.csv를 pandas를 이용해 bike_df의 이름으로 저장하고 행렬 개수와 5개의 자료를 뽑아보시오.

In [129]:
bike_df=pd.read_csv("C:\\Users\\LG-PC\\Desktop\\EXAMPLE\\bikeshare.csv")
print("행렬의 크기 :",bike_df.shape)
print("==============================================================")
print("상위 5 자료 예시")
print(bike_df.head())

행렬의 크기 : (10886, 12)
상위 5 자료 예시
              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  


### (2) bike_df에서 항목별 데이터 타입을 확인하는 코드를 작성하시오.

In [130]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


datetime 을 제외한 나머지 속성들은 모두 숫자형임을 확인하였습니다.

### (3) pandas를 이용해 datetime의 칼럼을 년, 월, 일, 시간과 같은 4개의 속성으로 분리하시오.
- 판다스에서 문자열을 datatime 타입으로 변환하는 apply(pd.to_datetime) 메서드를 이용해 datetime이라는 이름의 새로운 칼럼을 만드시오.

In [131]:
bike_df['datetime'] = pd.to_datetime(bike_df['datetime'])

- apply() 함수와 lambda를 이용해 새로운 datetime 칼럼에서 년, 월, 일, 시간 칼럼을 새로 만드시오.

- 방법 1

In [132]:
bike_df['year'] = bike_df['datetime'].dt.year 
bike_df['month'] = bike_df['datetime'].dt.month 
bike_df['day']= bike_df['datetime'].dt.day
bike_df['hour']=bike_df['datetime'].dt.hour

- 방법 2 : lambda 포함

In [133]:
# year = lambda x:x.dt.year
# bike_df ['year'] = year(bike_df['datetime'])
# month = lambda x:x.dt.month
# bike_df ['month'] = month(bike_df['datetime'])
# day = lambda x:x.dt.day
# bike_df ['day'] = day(bike_df['datetime'])
# hour = lambda x:x.dt.hour
# bike_df ['hour'] = hour(bike_df['datetime'])

- 3개의 bike_df를 뽑아 제대로 칼럼들이 생성되었는지 확인하시오.

In [134]:
bike_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2


* datatime, casual, registered 칼럼을 삭제 하시오.(casual + registered = count이므로 따로 필요하지 않습니다.) 

In [135]:
del bike_df['datetime']
del bike_df['casual']
del bike_df['registered']

In [136]:
bike_df.columns

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'count', 'year', 'month', 'day', 'hour'],
      dtype='object')

## 2. 선형 회귀 실습하기

### (1) X_fearues와 y_target이라는 이름으로 독립변수와 종속변수를 분리하시오. (종속변수 = count)

In [137]:
y_target=bike_df['count']
X_features=bike_df.drop(columns=['count'])

### (2) 숫자형 카테고리 값을 pandas의 get_dummies를 이용해 원-핫 인코딩하시오.
- 현재 데이터타입이 숫자형 카테고리인 칼럼은 year,month,day,hour,holiday,workingday,season,weather 입니다.
- X_features_ohe란 이름에 저장하세요.

In [146]:
X_features.columns

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'year', 'month', 'day', 'hour'],
      dtype='object')

In [147]:
X_features['year'] = X_features['year'].astype(str)
X_features['month'] = X_features['month'].astype(str)
X_features['day'] = X_features['day'].astype(str)
X_features['hour'] = X_features['hour'].astype(str)
X_features['holiday'] = X_features['holiday'].astype(str)
X_features['workingday'] = X_features['workingday'].astype(str)
X_features['season'] = X_features['season'].astype(str)
X_features['weather'] = X_features['weather'].astype(str)

X_features_ohe=pd.get_dummies(X_features, columns=['year', 'month','day','hour','holiday','workingday','season','weather'])

In [156]:
X_features_ohe.shape

(10886, 73)

### (3) 사이킷런에서 train_test_split을 이용해 훈련세트와 테스트 세트를 분리하시오.
- 이때 종속변수는 count 칼럼이고 독립변수는 종속변수 이외의 모든 칼럼입니다.
- X_train,X_test,y_train,y_test 이란 이름으로 세트를 만들고 test_size=0.3,random_state=0 이라는 옵션 추가하세요.

In [163]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_features_ohe, y_target, test_size=0.3, shuffle=True, random_state=0)

In [170]:
print("shape of X_train: ", X_train.shape)
print("shape of X_test: ", X_test.shape)
print("shape of y_train: ", y_train.shape)
print("shape of y_test: ", y_test.shape)

shape of X_train:  (7620, 73)
shape of X_test:  (3266, 73)
shape of y_train:  (7620,)
shape of y_test:  (3266,)


### (4) 회귀 종류별로 실습

#### i. 선형 회귀
- 사이킷런에서 LinearRegression 을 불러와 fit한 후 X_test를 예측하여 pred란 이름에 저장하시오.
- 사이킷런에서의 mean_square_error과 np.sqrt()를 이용해 y_test와 pred의 RMSE를 출력하시오.
- 이 모델의 intercept와 coefficient를 출력하시오.

In [189]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)
pred=lin_reg.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE: ", RMSE)
print("intercept: ",lin_reg.intercept_)
print("coefficient:")
print(lin_reg.coef_)

RMSE:  101.62065976440512
intercept:  851157451429.2461
coefficient:
[ 2.76972041e+00  1.85738393e+00 -8.03573634e-01 -7.33338844e-01
 -9.88080882e+11 -9.88080882e+11  2.12426173e+11  6.27171493e+11
  6.27171493e+11  6.27171493e+11  2.12426173e+11  2.12426173e+11
  1.01853305e+12  1.01853305e+12  1.01853305e+12  1.26550942e+12
  1.26550942e+12  1.26550942e+12  5.40508858e+10  5.40508859e+10
  5.40508859e+10  5.40508858e+10  5.40508859e+10  5.40508859e+10
  5.40508859e+10  5.40508859e+10  5.40508859e+10  5.40508858e+10
  5.40508859e+10  5.40508858e+10  5.40508859e+10  5.40508859e+10
  5.40508859e+10  5.40508858e+10  5.40508858e+10  5.40508858e+10
  5.40508858e+10  3.39880923e+09  3.39880921e+09  3.39880934e+09
  3.39880936e+09  3.39880940e+09  3.39880940e+09  3.39880938e+09
  3.39880939e+09  3.39880945e+09  3.39880961e+09  3.39880958e+09
  3.39880947e+09  3.39880919e+09  3.39880939e+09  3.39880933e+09
  3.39880930e+09  3.39880926e+09  3.39880919e+09  3.39880918e+09
  3.39880920e+09  3.3

#### ii. 확률적 경사 하강법을 이용한 선형 회귀
- 사이킷런에서 제공하는 SGDRegressor을 이용해 위와 같은 과정을 반복하시오.
- 이때 max_iter=1000,tol=1e-3,penalty=None,eta0=0.1 인 옵션을 추가하시오.

In [190]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=0)
sgd_reg.fit(X_train,y_train)
pred=sgd_reg.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE: ", RMSE)
print("intercept: ",sgd_reg.intercept_)
print("coefficient:")
print(sgd_reg.coef_)

RMSE:  6042715648950.735
intercept:  [-1.01217205e+11]
coefficient:
[-2.30420461e+11 -1.24274407e+11  7.35994453e+10 -1.06671031e+11
  5.86973166e+10 -1.59914522e+11 -8.75227103e+09  7.74002520e+10
 -3.93085574e+10 -1.57541709e+11 -2.96277392e+10  8.87709135e+10
  7.09500038e+10  8.66565359e+10 -7.89390929e+10  1.31194396e+10
  1.92310340e+10 -1.43176015e+11  1.30721449e+11 -2.36203588e+11
  1.41304608e+11  6.92515724e+10  1.73958066e+11  4.68677890e+10
 -3.07600025e+10 -1.89375474e+11 -1.29256668e+09  7.95104691e+10
 -5.98966649e+10  7.63170554e+10 -5.32146612e+10 -2.51532796e+11
  9.17823338e+10 -1.46450710e+11  1.14384766e+11 -1.32117256e+11
  7.55284049e+10 -9.34123094e+10  5.89423769e+10  1.86363613e+11
  2.64214713e+10  1.59910467e+11 -3.24481515e+10  1.21637023e+11
 -1.15325310e+11  2.08494374e+10  4.41759896e+10 -6.14807639e+10
 -1.62867940e+11  6.41416389e+10 -7.46280089e+10  2.04819988e+11
 -1.56675369e+11 -3.35458706e+10  2.36656441e+10 -1.36253446e+11
  1.09597599e+11 -2.22

RMSE가 너무 큰 것으로 보아, 데이터의 분포 및 scaling 과정이 필요해 보입니다. 

#### iv. 라쏘와 릿지
- 위와 같은 방식대로 사이킷런에서 Ridge와 Lasso를 불러와 RMSE를 출력하시오.
- 각각의 intercept, coefficient를 출력하시오.
- Ridge와 Lasso에서 alpha=0.1 이라는 옵션을 추가하시오.

In [196]:
from sklearn.linear_model import Ridge, Lasso

#릿지
print(" ")
print("RIDGE")
ridge_reg = Ridge(alpha=0.1, solver="cholesky", random_state=0)
ridge_reg.fit(X_train,y_train)
pred=ridge_reg.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE: ", RMSE)
print("intercept: ",ridge_reg.intercept_)
print("coefficient:")
print(ridge_reg.coef_)
print(" ")
print("LASSO")

#라쏘
lasso_reg = Lasso(alpha=0.1,random_state=0)
lasso_reg.fit(X_train,y_train)
pred=lasso_reg.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, pred))
print("RMSE: ", RMSE)
print("intercept: ",lasso_reg.intercept_)
print("coefficient:")
print(lasso_reg.coef_)

 
RIDGE
RMSE:  101.61963796032042
intercept:  93.5995147982032
coefficient:
[ 2.77635084e+00  1.85638810e+00 -8.04848466e-01 -7.32755619e-01
 -4.30350512e+01  4.30350512e+01 -2.78809868e+01  2.21827929e+01
  1.88035315e-01 -1.16829806e+00 -1.12490848e+01  6.11830405e+00
 -9.97383401e+00  1.52190326e+01  2.91904860e+00 -1.08898027e+01
 -1.00814072e+01  2.46162001e+01 -9.80464362e+00  1.02674291e+00
  2.07627610e+00 -5.06884674e+00  2.13600282e+00  2.81138083e+00
  6.74384093e+00  2.20615475e+00  1.81487720e+01 -6.33769375e+00
  1.18206464e+00 -2.81919690e+00  1.30083731e+00  8.68240255e-01
  2.18864828e+00  4.24340314e-01 -7.62089700e+00 -9.53377117e+00
  7.17480448e-02 -1.24689439e+02 -1.45447914e+02 -1.66084925e+01
  5.55252726e+00  5.22344891e+01  4.69859712e+01  3.27454723e+01
  3.70039430e+01  1.00703846e+02  2.62609572e+02  2.28766863e+02
  1.14486639e+02 -1.59260535e+02  3.37795043e+01 -1.94352514e+01
 -5.31216020e+01 -9.24930957e+01 -1.66295508e+02 -1.68110470e+02
 -1.49564140e+

## 3. 로지스틱 회귀 분석 실습 _ 와인 종류 분류하기 : 와인의 화학 조성을 사용하여 와인의 종류를 예측하기 위한 데이터이다. 

In [197]:
# 와인 데이터를 불러오기 위한 코드입니다. 꼭 실행해주세요.
from sklearn import datasets

ㅇ 타겟 데이터 : 와인의 종류 0, 1, 2의 세가지 값

ㅇ 특징 데이터

* 알콜(Alcohol)

* 말산(Malic acid)

* 회분(Ash)

* 회분의 알칼리도(Alcalinity of ash)

* 마그네슘(Magnesium)

* 총 폴리페놀(Total phenols)

* 플라보노이드 폴리페놀(Flavanoids)

* 비 플라보노이드 폴리페놀(Nonflavanoid phenols)

* 프로안토시아닌(Proanthocyanins)

* 색상의 강도(Color intensity)

* 색상(Hue)

* 희석 와인의 OD280/OD315 비율 (OD280/OD315 of diluted wines)

* 프롤린(Proline)

####3-1. 와인 데이터 가져오기 

In [198]:
wine_data = datasets.load_wine()

더 자세한 데이터에 대한 설명을 보고싶으시다면 아래의 코드를 실행해주시면 됩니다. 

In [199]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

#### 3-2. 입력변수와 목표변수 설정하기
* 입력변수를 사용하기 편하게 pandas dataframe으로 변환해주세요.
  - 입력변수는 data (.data) 함수를 써서 가져오시면 됩니다.
  - 칼럼명 가져오기 : .feature_names 
* 목표변수를 사용하기 편하게 pandas dataframe으로 변환해주세요
  - 목표변수는 target (.target) 함수를 써서 가져요오시면 됩니다. 
  - 칼럼명은 'target'으로 지정해주세요

In [212]:
# 입력 변수를 사용하기 편하게 pandas dataframe으로 변환
X = pd.DataFrame(wine_data.data,columns=wine_data.feature_names)
# 목표 변수를 사용하기 편하게 pandas dataframe으로 변환
y = pd.DataFrame(wine_data.target)

In [217]:
X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


#### 3-3. 훈련 데이터, 테스트 데이터 만들기
* train_test_split 함수를 사용해 테스트 셋의 비율이 0.2가 되도록 X_train, X_test, y_train, y_test 을 만들어주세요. random_state=5로 설정해주세요.
* 경고메세지를 없애도록 y_train = y_train.values.ravel() 를 실행해주세요

In [218]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=5)
y_train = y_train.values.ravel()

#### 3-4. 모델훈련
* 로지스틱회귀 함수에서 옵션을 solver='saga', max_iter=7500으로 설정해주세요.
* 학습데이터로 모델을 학습시켜주세요

In [226]:
# 필요한 라이브러리 import
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="saga", random_state=42,max_iter=7500)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=7500, random_state=42, solver='saga')

#### 3-5. 예측
* 테스트 데이터 셋으로 로지스틱 회귀 모델을 이용해 와인 데이터 분류를 예측해 결과를 y_test_predict 변수에 담아주세요
* score함수를 사용해 로지스틱 회귀 모델의 성능을 score 변수에 담아주세요
* y_test_predict와 score 결과를 확인해주세요.

In [227]:
# 로지스틱 회귀 모델를 이용해서 각 와인 데이터 분류를 예측함
y_test_predict=log_reg.predict(X_test)

# 로지스틱 회귀 모델의 성능 확인 (정확성 %를 리턴함)
score=log_reg.score(X_test,y_test)

# 결과확인 
y_test_predict, score

(array([0, 1, 0, 0, 2, 2, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 1, 2, 0, 1, 1, 0, 0, 2, 2, 0, 1, 2, 1, 1, 1, 2, 0,
        2, 1, 0, 2, 0, 1, 0, 1, 1, 0]),
 0.8148148148148148)

#### (선택) 3-6. GridSearchCV 복습

`LogisticRegression` 모델에 `param_grid`를 사용하여 GridSearchCV를 진행해주세요.
- cv = 5
- 추가적으로 코드를 작성하실 필요 없이, 골뱅이@@@ 부분의 코드를 완성해서 출력해주시면 됩니다. 

In [228]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV

# param_grid (수정하셔도 됩니다)
params = {
    'penalty':['l1', 'l2', 'elasticnet'],        # l1 is Lasso, l2 is Ridge
    'C': np.linspace(0.00002,1,100)
}

# GridSearch
grid_search = GridSearchCV(log_reg, param_grid=params, cv=5)
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=7500, random_state=42,
                                          solver='saga'),
             param_grid={'C': array([2.00000000e-05, 1.01208081e-02, 2.02216162e-02, 3.03224242e-02,
       4.04232323e-02, 5.05240404e-02, 6.06248485e-02, 7.07256566e-02,
       8.08264646e-02, 9.09272727e-02, 1.01028081e-01, 1.11128889e-01,
       1.21229697e-01, 1.31330505e-01, 1.41431313e-01, 1.51532121...
       7.67681414e-01, 7.77782222e-01, 7.87883030e-01, 7.97983838e-01,
       8.08084646e-01, 8.18185455e-01, 8.28286263e-01, 8.38387071e-01,
       8.48487879e-01, 8.58588687e-01, 8.68689495e-01, 8.78790303e-01,
       8.88891111e-01, 8.98991919e-01, 9.09092727e-01, 9.19193535e-01,
       9.29294343e-01, 9.39395152e-01, 9.49495960e-01, 9.59596768e-01,
       9.69697576e-01, 9.79798384e-01, 9.89899192e-01, 1.00000000e+00]),
                         'penalty': ['l1', 'l2', 'elasticnet']})

학습된 gridsearch 모델에 저장되어 있는 best parameter, best score 를 출력해주세요.

In [229]:
grid_search.best_params_, grid_search.best_score_

({'C': 0.11112888888888889, 'penalty': 'l2'}, 0.8884126984126983)